In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from base64 import b64encode

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

# Various queries based on the "Rescue Type and Preferred Dog Breeds Table" 
# provided by Grazioso Salvare
QUERIES = {
    "Water": {
        'breed': {"$in": ['Labrador Retriever Mix', 'Chesa Bay Retr Mix', 'Newfoundland Mix']},        
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Mountain": {
        'breed': {"$in": ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},        
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Disaster": {
        'breed': {"$in": ['Doberman Pinsch', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},        
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    },
    "Reset": {}
}

###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "mongoAAC"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object
cur = shelter.read(QUERIES['Reset'])
df = pd.DataFrame.from_records(cur)

# Create the 2nd (pie) chart
fig = px.pie(df, names='breed', title='Breeds')

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'logo.png' # replace with your own image
encoded_image = b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(className='row', style={'display' : 'flex', 'height': '100px'}, children=[
        html.A(href="https://www.snhu.edu", children=[            
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height='110px', width='140px'),
        ]),
        html.Center(html.B(html.H1('CS-340 Dashboard - Nicolas Blaisdell'))),
    ]),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    html.Hr(),
    html.Div([
        dcc.Dropdown(
            options=[
               {'label': 'No criteria (Select a filter...)', 'value': 'Reset'},
               {'label': 'Water Rescue', 'value': 'Water'},
               {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain'},
               {'label': 'Disaster/Individual Tracking', 'value': 'Disaster'},
            ],
            value='Reset',
            id='filter-type')
    ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        style_table={'height': '175px', 'overflowY': 'auto'},
        sort_action='native',
        page_size=25        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'width': '100%'},
             children=[
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={"width": "25%"}
            ),
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={"width": "75%"}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
    cur = shelter.read(QUERIES[filter_type])
    df = pd.DataFrame.from_records(cur)
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
         
    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Breeds')
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    return [
       dcc.Graph(            
           figure = fig
       )    
    ]
    

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    #FIXME: Add in the code for your geolocation chart
    #If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    loc = [dff.iloc[1,13],dff.iloc[1,14]]
    return [
        dl.Map(style={'height': '250px'}, center=loc, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=loc, children=[
                dl.Tooltip(dff.iloc[1,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app